In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
from scipy.linalg import eig
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RectWing.Models as rwm
import multiprocessing as mp

me.mechanics_printing()

In [2]:
def Model_Eigen_WT(model_num,b_modes,t_modes,save_num = None,linearise = False,calc_fixed_pos = True):
    if save_num == None:
        save_num = model_num
    dataset_name = 'WT'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 1.225
    p.s_t.value = 1.345
    p.c.value = 0.15
    p.EI.value = 61
    p.GJ.value = 200
    p.rho_t.value = 10
    p.e_0.value = 0
    p.e_1.value = 0
    p.ratio_fwt.value = 0.2
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(1,40,79))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    vars_ls.append((p.c_dmax,[0,1]))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))
    vars_ls.append((p.k_fwt,[0,1e4]))
    
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')

    variables = [k for k,v in vars_ls]
    if linearise:
        sm = sm.msubs(p.GetSubs(0,p.fp,ignore=variables)).linearise(p).msubs({f:0 for f in p.fp})
    
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed_pos)   
    print(f'M{model_num}-{dataset_name}: Complete')
    flutdf.to_pickle(f'Eigen_{b_modes}B{t_modes}T-M{save_num}_{dataset_name}.pkl')

In [3]:
def Model_Eigen_HALE(model_num,b_modes,t_modes,save_num = None,linearise = False,calc_fixed_pos = True):
    if save_num == None:
        save_num = model_num
    dataset_name = 'HALE'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 0.0889
    p.s_t.value = 16
    p.c.value = 1
    p.EI.value = 2e4
    p.GJ.value = 1e4
    p.rho_t.value = 0.75
    p.e_0.value = 0.25
    p.e_1.value = 0
    
    p.ratio_fwt.value = 0.2
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(1,40,79))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    vars_ls.append((p.c_dmax,[0,1]))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))
    vars_ls.append((p.k_fwt,[0,1e4]))
    
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    variables = [k for k,v in vars_ls]
    if linearise:
        sm = sm.msubs(p.GetSubs(0,p.fp,ignore=variables)).linearise(p).msubs({f:0 for f in p.fp})
    
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed_pos)   
    print(f'M{model_num}-{dataset_name}: Complete')
    flutdf.to_pickle(f'Eigen_{b_modes}B{t_modes}T-M{save_num}_{dataset_name}.pkl')

In [4]:
def Model_Eigen_JEC(model_num,b_modes,t_modes,save_num = None,linearise = False,calc_fixed_pos = True):
    if save_num == None:
        save_num = model_num
    
    dataset_name = 'JEC'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 1.225
    p.s_t.value = 12
    p.c.value = 1.8
    p.EI.value = 9.77e6
    p.GJ.value = 0.99e6
    p.rho_t.value = 19.53
    p.e_0.value = 0.08
    p.e_1.value = 0  
    p.ratio_fwt.value = 0.2
    
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    p.k_fwt.value = 0
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(1,100,100))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    vars_ls.append((p.c_dmax,[0,0.5,1]))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))
    vars_ls.append((p.k_fwt,[0,1e7]))
    
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    variables = [k for k,v in vars_ls]
    if linearise:
        sm = sm.msubs(p.GetSubs(0,p.fp,ignore=variables)).linearise(p).msubs({f:0 for f in p.fp})
    
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed_pos)   
    print(f'M{model_num}-{dataset_name}: Complete')
    flutdf.to_pickle(f'Eigen_{b_modes}B{t_modes}T-M{save_num}_{dataset_name}.pkl')

In [11]:
Model_Eigen_WT(4,3,3)

M4-WT: Started


IndexError: list index out of range

In [6]:
b_modes = 3
t_modes = 3
pool = mp.Pool(mp.cpu_count())

# running all the models
#for k in range(6):
#for k in [4,5]:
#    pool.apply_async(Model_Eigen_WT,args=(k,b_modes,t_modes))
#    pool.apply_async(Model_Eigen_JEC,args=(k,b_modes,t_modes))
#    pool.apply_async(Model_Eigen_HALE,args=(k,b_modes,t_modes))

## linearise the full model about the zero position
pool.apply_async(Model_Eigen_WT,args=(0,b_modes,t_modes,6,True))
pool.apply_async(Model_Eigen_JEC,args=(0,b_modes,t_modes,6,True))
pool.apply_async(Model_Eigen_HALE,args=(0,b_modes,t_modes,6,True))

## linearise the no ZFR about the zero position
pool.apply_async(Model_Eigen_WT,args=(1,b_modes,t_modes,7,True))
pool.apply_async(Model_Eigen_JEC,args=(1,b_modes,t_modes,7,True))
pool.apply_async(Model_Eigen_HALE,args=(1,b_modes,t_modes,7,True))


pool.close()
pool.join()

M0-JEC: Started
M0-HALE: Started
M0-WT: Started
M1-WT: Started
M1-HALE: Started
M1-JEC: Started
M1-WT: Complete
M1-HALE: Complete
M1-JEC: Complete
M0-WT: Complete
M0-HALE: Complete
M0-JEC: Complete


In [11]:
b_modes = 3
t_modes = 3
pool = mp.Pool(mp.cpu_count())
p,v = JEC(3,3)
for k in range(8):
    pool.apply_async(Model_Eigen,args=(0,'JEC',3,3,p,v))
pool.close()
pool.join()